In [1]:
#!/usr/bin/env python3

import numpy as np
from move_base_msgs.msg import MoveBaseActionGoal, MoveBaseActionFeedback
from sensor_msgs.msg import Imu
from geometry_msgs.msg import PoseArray, Twist
import actionlib

In [ ]:
class position():
    def __init__(self):
        self.x = 0
        self.y = 0
        self.z = 0
        self.ox = 0
        self.oy = 0
        self.oz = 0
        self.ow = 0

class imu_sample():
    def __init__(self):
        self.acc_x = None
        self.acc_y = None
        self.acc_z = None
        self.gyro_x = None
        self.gyro_y = None
        self.gyro_z = None



class path_MoRPI:
    def __init__(self):
        self.ground_truth_sub = rospy.Subscriber('/ground_truth', Odometry, self.ground_truth_position_callback)
        self.imu_sub = rospy.Subscriber('/imu/inertial', Imu, self.imu_callback)
        self.target_sub = rospy.Subscriber('/path', PoseArray, self.path_callback)
        self.move_base_pub = rospy.Publisher('/husky_control/goal', MoveBaseGoal, queue_size=10)
        self.imu_readings = np.array()

    def ground_truth_position_callback(self, data):
        self.position_husky.ox = data.pose.pose.orientation.x
        self.position_husky.oy = data.pose.pose.orientation.y
        self.position_husky.oz = data.pose.pose.orientation.z
        self.position_husky.ow = data.pose.pose.orientation.w
        self.position_husky.x = data.pose.pose.position.x
        self.position_husky.y = data.pose.pose.position.y
        self.position_husky.z = data.pose.pose.position.z
    
    def imu_callback(self, data):
        imu_temp = imu_sample()
        imu_temp.acc_x = data.linear_acceleration.x
        imu_temp.acc_y = data.linear_acceleration.y
        imu_temp.acc_z = data.linear_acceleration.z
        imu_temp.gyro_x = data.angular_velocity.x
        imu_temp.gyro_y = data.angular_velocity.y
        imu_temp.gyro_z = data.angular_velocity.z
        self.imu_readings = np.append(self.imu_readings, imu_temp)

    def path_callback(self, data):
        for i in data:
            move_base_goal = MoveBaseGoal()
            move_base_goal.target_pose.header.frame_id = 'map'
            move_base_goal.target_pose.pose.position.x = i.position.x
            move_base_goal.target_pose.pose.position.y = i.position.y
            move_base_goal.target_pose.pose.position.z = i.position.z
            move_base_goal.target_pose.pose.orientation.x = i.orientation.x
            move_base_goal.target_pose.pose.orientation.y = i.orientation.y
            move_base_goal.target_pose.pose.orientation.z = i.orientation.z
            move_base_goal.target_pose.pose.orientation.w = i.orientation.w
            self.move_base_pub.publish(move_base_goal)
            rospy.sleep(1)


    


